# STEP 1 — Case Folding

In [ ]:
import os
import pandas as pd

def case_folding(text):
    """Mengubah teks menjadi huruf kecil (lowercase)."""
    if pd.isna(text):
        return ""
    return text.lower()

def step1_casefolding(input_file="cleandataset.csv", save_to="fix_data"):
    """Melakukan case folding langsung dari file CSV tunggal."""
    os.makedirs(save_to, exist_ok=True)

    # Pastikan file ada
    if not os.path.exists(input_file):
        print(f"❌ File {input_file} tidak ditemukan.")
        return

    # Baca file CSV langsung (bukan dari folder)
    df = pd.read_csv(input_file)
    print(f"🔹 Melakukan Case Folding pada file: {input_file}")

    # Pastikan kolom tersedia
    if 'judul' not in df.columns or 'konten' not in df.columns:
        print("⚠️ File tidak memiliki kolom 'judul' dan 'konten'. Proses dihentikan.")
        return

    # Terapkan case folding
    df['judul'] = df['judul'].astype(str).apply(case_folding)
    df['konten'] = df['konten'].astype(str).apply(case_folding)

    # Simpan hasil ke folder fix_data
    output_path = os.path.join(save_to, "step1_casefolding.csv")
    df.to_csv(output_path, index=False, encoding='utf-8')
    print(f"✅ Step 1 selesai — hasil disimpan di: {output_path}")

if __name__ == "__main__":
    step1_casefolding()


# Cleaning

In [ ]:
import os
import re
import string
import pandas as pd

def cleaning(text):
    """Menghapus angka dan tanda baca dari teks."""
    if pd.isna(text):
        return ""
    text = re.sub(r'\d+', '', text)  # hapus angka
    text = text.translate(str.maketrans('', '', string.punctuation))  # hapus tanda baca
    return text.strip()

def step2_cleaning(input_file="fix_data/step1_casefolding.csv", output_file="fix_data/step2_cleaning.csv"):
    """Membersihkan teks dari angka dan tanda baca, hasil disimpan ke file baru."""
    os.makedirs(os.path.dirname(output_file), exist_ok=True)

    # Pastikan file input ada
    if not os.path.exists(input_file):
        print(f"❌ File {input_file} tidak ditemukan.")
        return

    print(f"🔹 Membaca file: {input_file}")
    df = pd.read_csv(input_file)

    # Pastikan kolom sesuai
    if 'judul' not in df.columns or 'konten' not in df.columns:
        print("⚠️ File tidak memiliki kolom 'judul' dan 'konten'. Proses dihentikan.")
        return

    print("🧹 Membersihkan teks (hapus angka & tanda baca)...")
    df['judul'] = df['judul'].astype(str).apply(cleaning)
    df['konten'] = df['konten'].astype(str).apply(cleaning)

    # Simpan hasil
    df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"✅ Step 2 selesai — hasil disimpan di: {output_file}")

if __name__ == "__main__":
    step2_cleaning()


# Tokenizing

In [ ]:
import os
import pandas as pd

def tokenizing(text):
    """Memecah teks menjadi token (berdasarkan spasi)."""
    if pd.isna(text):
        return []
    return text.split()

def step3_tokenizing(input_file="fix_data/step2_cleaning.csv", output_file="fix_data/step3_tokenizing.csv"):
    """Melakukan tokenisasi pada kolom 'judul' dan 'konten'."""
    os.makedirs(os.path.dirname(output_file), exist_ok=True)

    # Pastikan file input ada
    if not os.path.exists(input_file):
        print(f"❌ File {input_file} tidak ditemukan.")
        return

    print(f"🔹 Membaca file: {input_file}")
    df = pd.read_csv(input_file)

    # Pastikan kolom sesuai
    if 'judul' not in df.columns or 'konten' not in df.columns:
        print("⚠️ File tidak memiliki kolom 'judul' dan 'konten'. Proses dihentikan.")
        return

    print("✂️ Melakukan tokenisasi teks...")
    df['judul'] = df['judul'].astype(str).apply(tokenizing)
    df['konten'] = df['konten'].astype(str).apply(tokenizing)

    # Simpan hasil tokenisasi
    df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"✅ Step 3 selesai — hasil disimpan di: {output_file}")

if __name__ == "__main__":
    step3_tokenizing()


# Stopword Removal

In [ ]:
import os
import pandas as pd
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

# Inisialisasi daftar stopword Bahasa Indonesia
stop_factory = StopWordRemoverFactory()
stopwords_id = set(stop_factory.get_stop_words())

def remove_stopwords(tokens):
    """Menghapus stopword dari token list."""
    if isinstance(tokens, str):
        # Pastikan format list dari CSV diubah kembali ke list kata
        tokens = tokens.strip("[]").replace("'", "").split(", ")
    return [word for word in tokens if word not in stopwords_id]

def step4_stopword(input_file="fix_data/step3_tokenizing.csv", output_file="fix_data/step4_stopword.csv"):
    """Menghapus stopword dari kolom 'judul' dan 'konten'."""
    os.makedirs(os.path.dirname(output_file), exist_ok=True)

    # Pastikan file input ada
    if not os.path.exists(input_file):
        print(f"❌ File {input_file} tidak ditemukan.")
        return

    print(f"🔹 Membaca file: {input_file}")
    df = pd.read_csv(input_file)

    # Pastikan kolom sesuai
    if 'judul' not in df.columns or 'konten' not in df.columns:
        print("⚠️ File tidak memiliki kolom 'judul' dan 'konten'. Proses dihentikan.")
        return

    print("🧹 Menghapus stopword...")
    df['judul'] = df['judul'].apply(remove_stopwords)
    df['konten'] = df['konten'].apply(remove_stopwords)

    # Simpan hasilnya
    df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"✅ Step 4 selesai — hasil disimpan di: {output_file}")

if __name__ == "__main__":
    step4_stopword()


# Stemming

In [ ]:
import os
import pandas as pd
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# Inisialisasi stemmer Bahasa Indonesia
stemmer = StemmerFactory().create_stemmer()

def stemming(tokens):
    """Melakukan stemming dan hasilnya dikembalikan dalam bentuk token list."""
    if isinstance(tokens, str):
        tokens = tokens.strip("[]").replace("'", "").split(", ")
    text = ' '.join(tokens)
    stemmed_text = stemmer.stem(text)
    return stemmed_text.split()  # hasil akhirnya berupa list kata

def step5_stemming(input_file="fix_data/step4_stopword.csv", 
                   output_file="fix_data/step5_stemming_token.csv", 
                   n_rows=5):
    """Melakukan stemming pada kolom 'judul' dan 'konten' (5 baris pertama, hasil token)."""
    os.makedirs(os.path.dirname(output_file), exist_ok=True)

    # Pastikan file input ada
    if not os.path.exists(input_file):
        print(f"❌ File {input_file} tidak ditemukan.")
        return

    print(f"🔹 Membaca file: {input_file}")
    df = pd.read_csv(input_file)

    # Ambil hanya beberapa baris pertama untuk tes cepat
    df = df.head(n_rows)
    print(f"🧪 Memproses {len(df)} baris pertama untuk pengujian...")

    # Pastikan kolom sesuai
    if 'judul' not in df.columns or 'konten' not in df.columns:
        print("⚠️ File tidak memiliki kolom 'judul' dan 'konten'. Proses dihentikan.")
        return

    print("🌾 Melakukan stemming pada teks (judul & konten)...")
    df['judul'] = df['judul'].apply(stemming)
    df['konten'] = df['konten'].apply(stemming)

    # Simpan hasil ke file baru
    df.to_csv(output_file, index=False, encoding='utf-8')
    print(f"✅ Step 5 selesai — hasil disimpan di: {output_file}")

if __name__ == "__main__":
    step5_stemming()


In [ ]:
import os
import pandas as pd
from multiprocessing import Pool, cpu_count
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

# === Inisialisasi stemmer Bahasa Indonesia (Sastrawi) ===
stemmer = StemmerFactory().create_stemmer()

def stemming_worker(text):
    """Worker function untuk multiprocessing — melakukan stemming pada satu teks."""
    if isinstance(text, str):
        text = text.strip("[]").replace("'", "").split(", ")
    joined = ' '.join(text)
    stemmed = stemmer.stem(joined)
    return stemmed.split()  # hasil tetap berupa token list

def process_batch(df_batch, num_cores):
    """Memproses satu batch data dengan multiprocessing."""
    with Pool(num_cores) as pool:
        judul_stem = pool.map(stemming_worker, df_batch['judul'])
        konten_stem = pool.map(stemming_worker, df_batch['konten'])
    df_batch['judul'] = judul_stem
    df_batch['konten'] = konten_stem
    return df_batch

def step5_stemming_parallel_batch(
        input_file="fix_data/step4_stopword.csv",
        output_file="fix_data/step5_stemming_token_parallel_batch.csv",
        batch_size=10000):

    """Melakukan stemming paralel per batch untuk efisiensi memori."""
    os.makedirs(os.path.dirname(output_file), exist_ok=True)

    # Pastikan file input tersedia
    if not os.path.exists(input_file):
        print(f"❌ File {input_file} tidak ditemukan.")
        return

    num_cores = max(1, cpu_count() - 1)
    print(f"🧠 Menggunakan {num_cores} core CPU untuk stemming paralel...")
    print(f"⚙️ Ukuran batch: {batch_size} baris per iterasi\n")

    # Hitung total baris di file dengan encoding utf-8
    try:
        with open(input_file, encoding='utf-8') as f:
            total_rows = sum(1 for _ in f) - 1  # dikurangi header
    except UnicodeDecodeError:
        # Jika file pakai encoding lain (misal cp1252), fallback otomatis
        with open(input_file, encoding='latin1') as f:
            total_rows = sum(1 for _ in f) - 1

    print(f"📊 Total baris dalam dataset: {total_rows}")

    # Baca dan proses per batch (pakai encoding aman)
    batch_iter = pd.read_csv(input_file, chunksize=batch_size, encoding='utf-8', on_bad_lines='skip')
    is_first = True

    for i, df_chunk in enumerate(batch_iter, start=1):
        start_idx = (i - 1) * batch_size + 1
        end_idx = start_idx + len(df_chunk) - 1
        print(f"\n🔹 Memproses batch {i}: baris {start_idx:,}–{end_idx:,} ...")

        if 'judul' not in df_chunk.columns or 'konten' not in df_chunk.columns:
            print("⚠️ File tidak memiliki kolom 'judul' dan 'konten'. Batch dilewati.")
            continue

        df_processed = process_batch(df_chunk, num_cores)

        # Simpan ke file (append mode)
        df_processed.to_csv(output_file, mode='a', index=False, header=is_first, encoding='utf-8')
        is_first = False  # hanya tulis header di batch pertama

        print(f"✅ Batch {i} selesai dan disimpan ke: {output_file}")

    print("\n🎉 Semua batch selesai diproses dan disimpan.")
    print(f"📁 File akhir: {output_file}")

if __name__ == "__main__":
    step5_stemming_parallel_batch()
